DeepFace analyze method wraps age, gender and race prediction models. Those models build VGG-Face first, modify its output layer and load pre-trained weights for those sub models. Each model size is 500 MB.

However, we can build VGG-Face once and find its early layer output once. Then, transfer this early layer output to 3 layer basic network. In this way, the size of the all age, gender and race model would decrease from 500 MB to 2 MB. 

This notebook makes those sub models into smaller ones.

In [1]:
import numpy as np
import tensorflow as tf
import keras
from keras.preprocessing import image
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.layers import Dense, Activation, Dropout, Flatten, Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Activation
from keras.layers import Conv2D, AveragePooling2D
from keras.models import Model, Sequential
import keras.backend as K

Using TensorFlow backend.


In [2]:
from deepface import DeepFace
from deepface.commons import functions
from deepface.extendedmodels import Age

## VGG-Face

In [3]:
#the both if and else blocks work well
if True:
    model = DeepFace.build_model('VGG-Face')
else:
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Convolution2D(4096, (7, 7), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Convolution2D(4096, (1, 1), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Convolution2D(2622, (1, 1)))
    model.add(Flatten())
    model.add(Activation('softmax'))

    model.load_weights('C:/Users/IS96273/.deepface/weights/vgg_face_weights.h5')

## V1 model

In [4]:
if True:
    classes = 101

    base_model_output = Sequential()
    base_model_output = Convolution2D(classes, (1, 1), name='predictions')(model.layers[-4].output)
    base_model_output = Flatten()(base_model_output)
    base_model_output = Activation('softmax')(base_model_output)

    age_model = Model(inputs=model.input, outputs=base_model_output)

    age_model.load_weights("C:/Users/IS96273/.deepface/weights/age_model_weights.h5")
else:
    #else block causes trouble. I cannot understand why.
    age_model = DeepFace.build_model('Age')

## V2 model

In [5]:
common_model = Model(inputs = model.input, outputs = model.layers[-4].output)

In [6]:
age_model_v2 = Sequential()
age_model_v2.add(Convolution2D(101, (1, 1), input_shape=(1, 1, 4096)))
age_model_v2.add(Flatten())
age_model_v2.add(Activation('softmax'))

In [7]:
if True:
    for i in range(0, 3):
        age_model_v2.layers[i].set_weights(age_model.layers[-3+i].get_weights())
    age_model_v2.save_weights("age_model_v2_weights.h5")
else:
    age_model_v2.load_weights("age_model_v2_weights.h5")

## Test

In [8]:
img_path = "deepface/tests/dataset/img1.jpg"

In [9]:
img = functions.preprocess_face(img_path)

In [10]:
img.shape

(1, 224, 224, 3)

In [11]:
probas = age_model.predict(img)[0]
print("v1 result: ", Age.findApparentAge(probas))

v1 result:  31.803729148267408


In [12]:
common_output = common_model.predict(img)
probas_v2 = age_model_v2.predict(common_output)
print("v2 result: ", Age.findApparentAge(probas_v2))

v2 result:  31.803729148267408
